## Импорты

In [ ]:
import time
import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import pandas as pd

## Сбор данных о клубах

In [ ]:
clubs_list = dict()
ligues_pairs = [
    ['premier-league', 'GB1'],
    ['ligue-1', 'FR1'],
    ['serie-a', 'IT1'],
    ['bundesliga', 'L1'],
    ['laliga', 'ES1']
]
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

for pair in ligues_pairs:
    for year in years:
        time.sleep(2)
        if pair[0] not in clubs_list:
            clubs_list[pair[0]] = dict()
        all_teams = {}
        url = f'https://www.transfermarkt.com/{pair[0]}/startseite/wettbewerb/{pair[1]}/plus/?saison_id={year}'
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            squad_table = soup.find('table', {'class': 'items'})
            rows = squad_table.find_all('tr', {'class': ['odd', 'even']})
            for row in rows:
                club_name = row.find('td', {'class': 'hauptlink'}).text.strip()
                mean_age = float(row.find_all('td')[-4].text.replace(',', '.'))
                num_foreigns = float(row.find_all('td')[-3].text.replace(',', '.'))
                price = 1000 * float(row.text.split('€')[-1][:-2]) if row.text.split('€')[-1][-1] == 'n'\
                 else float(row.text.split('€')[-1][:-1])
                all_teams[club_name] = [mean_age, num_foreigns, price]
        clubs_list[pair[0]][year] = dict()
        clubs_list[pair[0]][year] = all_teams
        print(clubs_list)

In [ ]:
club_list_final = []
for ligue in clubs_list:
    for year in clubs_list[ligue]:
        for club in clubs_list[ligue][year]:
            club_list_final.append([ligue, year, club, *clubs_list[ligue][year][club]])

In [ ]:
# pd.DataFrame(
#     club_list_final,
#     columns = ['ligue', 'year', 'club', 'mean_age', 'num_foreigns', 'price']
#     ).to_csv('Clubs_prices_2017-2024.csv')

## Сбор данных об игроках

In [ ]:
players_list = {}

for pair in ligues_pairs:
    for year in tqdm(years):
        time.sleep(2)
        if pair[0] not in players_list:
            players_list[pair[0]] = dict()
        players_list[pair[0]][year] = dict()
        url = f'https://www.transfermarkt.com/{pair[0]}/startseite/wettbewerb/{pair[1]}/plus/?saison_id={year}'
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('td', {'class': 'zentriert no-border-rechts'})
        for link in tqdm(links[:(len(links) // 2)]):
            test_url = str(link).split('"')[3]
            test_url = test_url.replace('spielplan', 'startseite')
            test_url = test_url.replace(f'saisod_id/{year}', f'plus/0/galerie/0?saison_id={year}')
            final_url = 'https://www.transfermarkt.com' + test_url

            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(final_url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                squad_table = soup.find('table', {'class': 'items'})
                rows = squad_table.find_all('tr', {'class': ['odd', 'even']})
                for row in rows:
                    club_name = soup.text.split('- Club profile')[0].split('\n')[-1].strip()
                    player_name = row.find('td', {'class': 'hauptlink'}).text.strip()
                    price = row.text.split('€')[1] if len(row.text.split('€')) > 1 else '-'
                    countries = [str(i).split('"')[1] for i in row.find_all('td', {'class': 'zentriert'})[-1]\
            .find_all('img', {'class': 'flaggenrahmen'})]
                    players_list[pair[0]][year][(club_name, player_name)] = [price, countries, row]

In [ ]:
players_list_final = []
for ligue in players_list:
    for year in players_list[ligue]:
        for player in players_list[ligue][year]:
            players_list_final.append([
                ligue,
                year,
                player[0],
                player[1],
                players_list[ligue][year][player][0],
                ])

In [ ]:
players_pd = pd.DataFrame(
    players_list_final,
    columns = ['ligue', 'year', 'club', 'name', 'price']
)

In [ ]:
players_pd.to_csv('Players_prices_2017-2024.csv')